# Setup

Inspired by the code in this [tutorial](https://medium.com/towards-formula-1-analysis/formula-1-data-analysis-tutorial-2021-russian-gp-to-box-or-not-to-box-da6399bd4a39)

In [1]:
import fastf1 as ff1 
from fastf1 import plotting 
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.collections import LineCollection
from matplotlib import cm
import numpy as np
import pandas as pd

In [2]:
# Setup plotting
plotting.setup_mpl()

# Enable the cache
ff1.Cache.enable_cache('../cache') 

# Get rid of some pandas warnings that are not relevant for us at the moment
pd.options.mode.chained_assignment = None

# Race

In [3]:
race = ff1.get_session(2021,'Spain', 'R')

In [7]:
laps = race.load_laps(with_telemetry=True)

core           INFO 	Loading laps for Spanish Grand Prix - Race [v2.1.13]
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api         WARNING 	Driver 55: Ignoring late data for a previously processed lap.The data may contain errors (previous: 28; current 29)
api         WARNING 	Driver 22: Ignoring late data for a previously processed lap.The data may contain errors (previous: 2; current 3)
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for timing_app_data. Loading data...
api            INFO 	Fetching timing app data...
api            INFO 	Data has been written to cache!
core           INFO 	Processing timing data...
api            INFO 	No cached data found for driver_info. Loading data...
api            INFO 	Fetching driver list...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data 

In [8]:
# Calculate RaceLapNumber (LapNumber minus 1 since the warmup lap is included in LapNumber)
laps['RaceLapNumber'] = laps['LapNumber'] - 1

# Starting from lap 45 it started raining
laps = laps.loc[laps['RaceLapNumber'] >= 45]

In [10]:
laps

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate,RaceLapNumber
45,0 days 01:39:17.778000,44,0 days 00:01:20.730000,46,3,NaT,NaT,0 days 00:00:22.925000,0 days 00:00:30.377000,0 days 00:00:27.428000,...,MEDIUM,10.0,False,0 days 01:37:57.048000,Mercedes,HAM,1,True,2021-05-09 14:07:57.056,45
46,0 days 01:40:39.126000,44,0 days 00:01:21.348000,47,3,NaT,NaT,0 days 00:00:22.857000,0 days 00:00:30.610000,0 days 00:00:27.881000,...,MEDIUM,11.0,False,0 days 01:39:17.778000,Mercedes,HAM,1,True,2021-05-09 14:09:17.786,46
47,0 days 01:42:00.396000,44,0 days 00:01:21.270000,48,3,NaT,NaT,0 days 00:00:22.556000,0 days 00:00:30.746000,0 days 00:00:27.968000,...,MEDIUM,12.0,False,0 days 01:40:39.126000,Mercedes,HAM,1,True,2021-05-09 14:10:39.134,47
48,0 days 01:43:21.551000,44,0 days 00:01:21.155000,49,3,NaT,NaT,0 days 00:00:22.978000,0 days 00:00:30.448000,0 days 00:00:27.729000,...,MEDIUM,13.0,False,0 days 01:42:00.396000,Mercedes,HAM,1,True,2021-05-09 14:12:00.404,48
49,0 days 01:44:42.545000,44,0 days 00:01:20.994000,50,3,NaT,NaT,0 days 00:00:22.924000,0 days 00:00:30.614000,0 days 00:00:27.456000,...,MEDIUM,14.0,False,0 days 01:43:21.551000,Mercedes,HAM,1,True,2021-05-09 14:13:21.559,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241,0 days 02:01:32.879000,9,0 days 00:01:27.116000,60,3,NaT,NaT,0 days 00:00:23.815000,0 days 00:00:32.468000,0 days 00:00:30.833000,...,MEDIUM,18.0,True,0 days 02:00:05.763000,Haas F1 Team,MAZ,1,True,2021-05-09 14:30:05.771,59
1242,0 days 02:03:02.926000,9,0 days 00:01:30.047000,61,3,NaT,NaT,0 days 00:00:24.428000,0 days 00:00:33.151000,0 days 00:00:32.468000,...,MEDIUM,19.0,True,0 days 02:01:32.879000,Haas F1 Team,MAZ,1,True,2021-05-09 14:31:32.887,60
1243,0 days 02:04:30.325000,9,0 days 00:01:27.399000,62,3,NaT,NaT,0 days 00:00:24.252000,0 days 00:00:33.727000,0 days 00:00:29.420000,...,MEDIUM,20.0,True,0 days 02:03:02.926000,Haas F1 Team,MAZ,1,True,2021-05-09 14:33:02.934,61
1244,0 days 02:05:55.064000,9,0 days 00:01:24.739000,63,3,NaT,NaT,0 days 00:00:23.814000,0 days 00:00:32.169000,0 days 00:00:28.756000,...,MEDIUM,21.0,True,0 days 02:04:30.325000,Haas F1 Team,MAZ,1,True,2021-05-09 14:34:30.333,62


In [9]:
drivers = pd.unique(laps['Driver'])

In [ ]:
laps.pick_driver

In [12]:
telemetry = pd.DataFrame()

#telemetry is only retrieved driver-by-driver

for lap in drivers:
    driver_laps = laps.pick_driver(driver)